In [ ]:
readFiles = False
%run ./init.ipynb

In [ ]:
def in_notebook():
    """
    Returns ``True`` if the module is running in IPython kernel,
    ``False`` if in IPython shell or other Python shell.
    """
    return 'ipykernel' in sys.modules

def ipython_info():
    ip = False
    if 'ipykernel' in sys.modules:
        ip = 'notebook'
    elif 'IPython' in sys.modules:
        ip = 'terminal'
    return ip

In [ ]:
def renameFileIfExist(fileName):
    if os.path.exists(fileName):
        os.rename(fileName,'old_' + fileName)

In [ ]:
listing = {}

In [ ]:
%%time

startRange = 1
stopRange = 101

print(ipython_info()) 

if (ipython_info() == 'terminal'):
    if len(sys.argv) > 0:
        print('in sys if statement')
        startRange = int(sys.argv[1])
        stopRange = int(sys.argv[2])
        print('start range is ' + str(startRange))
        print('stop range is ' + str(stopRange))

rowAttributes = {}
refStartx = startRange

blankOutputList = []
for i in manualColumns:
    blankOutputList.append('')
print('startRange = ' + str(startRange) + ' and stopRange = ' + str(stopRange))
for x in range(startRange, stopRange):
    #rowAttributes = {}
    scraper = cloudscraper.create_scraper(
        delay=.7,
        browser={
            'browser': 'firefox',
            'platform': 'windows',
            'mobile': False
        })
    
    html = scraper.get("https://niftys.com/nft/0x39ceaa47306381b6d79ad46af0f36bc5332386f2/" + str(x)).text
    soup = BeautifulSoup(html, 'lxml')
    rows = soup.find_all('div', 'label')
    for row in rows:
        cells = row
        str_row = str(row)
        value = str_row.split('!-- -->')
        cleanTextAttribute = BeautifulSoup(value[0], "lxml").get_text()
        if cleanTextAttribute == 'Available on PALM':
            continue
            
        if (len(value) > 1):
            cleanTextValue = BeautifulSoup(value[1], "lxml").get_text().strip()
        else:
            cleanTextValue = ''
        rowAttributes[cleanTextAttribute] = cleanTextValue
    
    if len(rows) == 0:
        output_list = blankOutputList.copy()
    else:
        output_list = [(rowAttributes[key]) for key in (manualColumns) if key in rowAttributes]
    
    listing[str(x)] = output_list
    #clear_output(wait=True)
    print(x)
    if(x % 5000 == 0):
        if refStartx == 1:
            refStartx = 0
        data_dictionary =  pd.DataFrame.from_dict(listing, orient='index', columns=manualColumns)  #pd.DataFrame(listing, orient='index')
        fileName = 'Attributes_' + str(refStartx + 1) + '_' + str(x) + '.csv'
        renameFileIfExist(fileName)
        data_dictionary.to_csv(fileName)
        refStartx = x
        listing.clear()

In [ ]:
#fileName = 'Attributes_' + str(refStartx + 1) + '_' + str(x) + '.csv'
#renameFileIfExist(fileName)
#data_dictionary =  pd.DataFrame.from_dict(listing, orient='index', columns=manualColumns)  #pd.DataFrame(listing, orient='index')
#data_dictionary.to_csv(fileName)

In [ ]:
#for count in range(0, len(manualColumns)):
#    print(count)
#    print(data_dictionary[manualColumns[count]].value_counts())
#    data_dictionary[manualColumns[count]].value_counts(normalize=True)
#    print('---------------------------------')

In [ ]:
print('done')